In [ ]:
# I worry that all of these xpath links will get broken by the react they're using. They had strange names.

# chrome://version/
#profile_path = r"C:\Users\Pingu\AppData\Local\Google\Chrome\User Data\Default"
#profile_name="Default"

# Options for selenium
import undetected_chromedriver as uc
from undetected_chromedriver import By
from typing import Union
import pandas as pd
import time
import json

options = uc.ChromeOptions()

# options.add_argument("--window-size=400,600")
# options.add_argument("--log-level=3")
# options.add_argument(f"--user-data-dir={profile_path}")
# options.add_argument(f"--profile-directory={profile_name}")
# options.add_argument("--auto-open-devtools-for-tabs")

driver = uc.Chrome(use_subprocess=False, options=options)

# Go to udemy
driver.get("https://www.udemy.com/")

Manually log into Udemy now because trying to log in automatically with 2FA and everything will be a pain in the ass. Save the cookies to skip this step later.

I tried to export my cookies from Chrome and load them into here before, but it didn't work. So be it.

In [ ]:
# To save cookies
saved_cookies = driver.get_cookies()
with open("cookies.json", "w") as file:
    json.dump(saved_cookies, file)

If you're restarting now you can use this instead of logging in again

In [ ]:
# To load in cookies
with open("cookies.json", "r") as file:
    saved_cookies = json.load(file)
for x in saved_cookies:
    driver.add_cookie(x)
# Wait for one second just to be sure (probably not needed)
time.sleep(1)
driver.get("https://www.udemy.com/dashboard/purchase-history/")
# Wait for the page to load (it can take a moment)
time.sleep(20)

In [ ]:
# Function to check if there are multiple courses in a purchase
def check_multiple_courses(web_element: uc.WebElement) -> Union[bool, uc.WebElement]:
    try:
        temp = web_element.find_element(By.XPATH, ".//button[@class='ud-btn ud-btn-medium ud-btn-link ud-heading-sm show-more_focusable-label__dmEhW']")
        return(temp)
    except:
        return(False)

In [ ]:
# Function to check if a refund button exists for a course
def check_refund(web_element: uc.WebElement) -> Union[bool, uc.WebElement]:
    try:
        temp = web_element.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text__JKVm6']")
        return(temp)
    except:
        return(False)

In [ ]:
# Items to store the purchases
purchases_df = pd.DataFrame(columns =["Date", "Purchase", "Price", "Receipt", "Invoice"])
courses_df = pd.DataFrame(columns =["Date", "Purchase", "Price", "Receipt", "Invoice", "Course"])

purchases_list = []
courses_list = []

# Get the purchases
purchase_table = driver.find_element(By.XPATH, "//table[@class='paginated-table_purchase-history-table__ncCll']")
purchases = purchase_table.find_elements(By.XPATH, ".//tr[@class='paginated-table_purchase-history-table--main-row__5hHCb']")

for ind, purchase in enumerate(purchases):
    # Get the purchase
    temp_purchase = purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text-container__EvFRz']")
    # Get the receipt
    receipt = purchase.find_element(By.XPATH, ".//span[@class='ud-btn-label' and contains(., 'Receipt')]")
    receipt = receipt.find_element(By.XPATH, "..")
    receipt = receipt.get_attribute("href")
    receipt = receipt.split("/")[-2]
    # Get the invoice
    invoice = purchase.find_element(By.XPATH, ".//span[@class='ud-btn-label' and contains(., 'Invoice')]")
    invoice = invoice.find_element(By.XPATH, "..")
    invoice = invoice.get_attribute("href")
    invoice = invoice.split("/")[-2]
    # Total price
    total_price = purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-history-table--data-cell__LO_7L']/div[@class='app_positive-text___Wb07']").text
    # Get the data ( I couldn't also done find_elements and selected the index 1, but this should be better and was neat to learn)
    purchase_date = purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-history-table--data-cell__LO_7L' and not(descendant::*)]").text

    # Add the purchase
    cur_purchase = {"Date": purchase_date, 
                    "Purchase": ind,
                    "Price": total_price,
                    "Receipt": receipt,
                    "Invoice": invoice}
    purchases_list.append(cur_purchase)
    print(purchases_list)

    print("=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
    print(temp_purchase.text)
    print(receipt)
    print(invoice)
    print(total_price)
    print(purchase_date)
    print("======================")

    # Check if it has multiple purchases
    temp_multiple = check_multiple_courses(temp_purchase)
    if temp_multiple != False:
        # Click open the courses
        temp_multiple.click()
        # Wait for it to load because it can take a moment
        time.sleep(1)
        # Get the individual courses
        courses = purchase_table.find_elements(By.XPATH, ".//tr[@class='paginated-table_purchase-history-table--sub-row__2g87b']")
        print(courses)
        for course in courses:
            # Check if it has a refund button
            course_refund = check_refund(course)
            if course_refund != False:
                course_name = course.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text__JKVm6 paginated-table_purchase-details--content-title-text__DxDuf ud-text-md']")
                course_name = course_name.find_element(By.XPATH, ".//a")
                c_name = course_name.get_attribute("innerHTML")
                c_link = course_name.get_attribute("href")
                course_price = course.find_element(By.XPATH, ".//div[@class='app_positive-text___Wb07']").text
                # Add the course
                cur_course = {"Date": purchase_date, 
                                "Purchase": ind,
                                "Price": total_price,
                                "Receipt": receipt,
                                "Invoice": invoice,
                                "Course": c_name,
                                "Link": c_link}
                courses_list.append(cur_course)
        # Close what we opened because the subrows are inserted directly into it
        temp_multiple.click()
    # If it's a singular course
    else:
        # Check for refund button
        try:
            course_refund = temp_purchase.find_element(By.XPATH, ".//div[@data-testid='redirect-button']")
            course_name = temp_purchase.find_element(By.XPATH, ".//div[@class='paginated-table_purchase-details--text__JKVm6 paginated-table_purchase-details--content-title-text__DxDuf ud-text-md']")
            c_name = course_name.find_element(By.XPATH, ".//a").text
            c_link = course_name.get_attribute("href")
            # Add the course
            cur_course = {"Date": purchase_date, 
                            "Purchase": ind,
                            "Price": total_price,
                            "Receipt": receipt,
                            "Invoice": invoice,
                            "Course": c_name,
                            "Link": c_link}
            courses_list.append(cur_course)
        except:
            pass

In [ ]:
print(purchases_list)
print(courses_list)